In [4]:
# To unzip the datasets nii.gz. However, it is not necessory to unzip them because the libriry 'nibabel' below will process zipped data implicitly.
# But I did not delete this part, maybe you can learn something in it.
import os
import gzip
import shutil

def decompress_gz_files(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith('.gz'):
                gz_file_path = os.path.join(root, file)
                output_file_path = os.path.splitext(gz_file_path)[0]
                # print(f"Unzipping: {gz_file_path} -> {output_file_path}")
                with gzip.open(gz_file_path, 'rb') as gz_file:
                    with open(output_file_path, 'wb') as output_file:
                        shutil.copyfileobj(gz_file, output_file)

data_dir = r'./brainweb_petmr_v2'
decompress_gz_files(data_dir)

In [2]:
# Output info of dataset
import nibabel as nib

# load NIfTI files
data_dir = r'./brainweb_petmr_v2/subject04/image_0.nii.gz'
image = nib.load(data_dir)

# get file data
image_data = image.get_fdata()

# check header info
header = image.header
print(header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 220 220 184   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1. 1. 1. 1. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 1.0
qoffset_x       : -0.0
qoffset_y       : -0.0
qoffset_z       : 0.0
srow_x          : [-1. -0. -0. -0.]
srow_y          :

In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# load NIfTI files
data_dir = r'./brainweb_petmr_v2/subject04/image_1.nii.gz'
# data_dir = r'./brainweb_petmr_v2/subject04/image_0.nii.gz'
image = nib.load(data_dir)
image_data = image.get_fdata()  # Extract data as NumPy array

# ensure image is 3D
# assert image_data.ndim == 3, "error!"

num_slices = image_data.shape[2]  # assume z-axis as slice direction

def show_nii_slice(slice_num):
    plt.figure(figsize=(8, 8))
    plt.imshow(image_data[:, :, slice_num], cmap='gray')
    plt.title(f'NIfTI Slice {slice_num + 1}')
    plt.axis('off')
    plt.show()

interact(show_nii_slice, slice_num=IntSlider(min=0, max=num_slices - 1, step=1, value=num_slices // 2))

interactive(children=(IntSlider(value=92, description='slice_num', max=183), Output()), _dom_classes=('widget-…

<function __main__.show_nii_slice(slice_num)>

In [3]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# load NIfTI files
data_dir = r'./brainweb_petmr_v2/subject04/image_1.nii.gz'
# data_dir = r'./brainweb_petmr_v2/subject04/image_0.nii.gz'
image = nib.load(data_dir)
image_data = image.get_fdata()  # Extract data as NumPy array

# ensure image is 3D
# assert image_data.ndim == 3, "error!"

# add poisson noise
noisy_image_data = np.random.poisson(image_data)

num_slices = noisy_image_data.shape[2]

def show_noisy_nii_slice(slice_num):
    plt.figure(figsize=(8, 8))
    plt.imshow(noisy_image_data[:, :, slice_num], cmap='gray')
    plt.title(f'Noisy NIfTI Slice {slice_num + 1}')
    plt.axis('off')
    plt.show()

interact(show_noisy_nii_slice, slice_num=IntSlider(min=0, max=num_slices - 1, step=1, value=num_slices // 2))

interactive(children=(IntSlider(value=92, description='slice_num', max=183), Output()), _dom_classes=('widget-…

<function __main__.show_noisy_nii_slice(slice_num)>

In [4]:
# create training dataset
import os
import nibabel as nib
import numpy as np

# Set the data directory
data_dir = './brainweb_petmr_v2'

# Initialize an empty list to store slice data
all_slices = []

# Iterate over each subject folder (assuming each folder starts with "subject")
for subject_folder in os.listdir(data_dir):
    subject_path = os.path.join(data_dir, subject_folder)
    if os.path.isdir(subject_path) and subject_folder.startswith('subject'):
        # For each subject, load image_0 and image_1
        for image_file in ['image_0.nii.gz', 'image_1.nii.gz']:
            image_path = os.path.join(subject_path, image_file)
            if os.path.exists(image_path):
                # Load the image file
                image = nib.load(image_path)
                image_data = image.get_fdata()  # Extract as a NumPy array
                
                # Get the center slice index
                center_slice = image_data.shape[2] // 2
                
                # Determine the range of slices (center ± 14 slices, total 35 slices)
                start_slice = max(0, center_slice - 14)
                end_slice = min(image_data.shape[2], center_slice + 15)  # Exclusive
                
                # Extract the slices and add them to all_slices
                slices = image_data[:, :, start_slice:end_slice]
                all_slices.extend([slices[:, :, i] for i in range(slices.shape[2])])

# Combine all slices into a single NumPy array
all_slices_array = np.array(all_slices)

# Add a new dimension to the data
all_slices_array = np.expand_dims(all_slices_array, axis=1)  # Shape: (N, 1, 220, 220)

# Ensure the data is correct
print(f"Total dataset shape: {all_slices_array.shape}")

# Save as a .npy file
os.makedirs('./PET_data/Train_npy', exist_ok=True)
np.save('./PET_data/Train_npy/training_sinograms_noisefree.npy', all_slices_array)

print("Data has been successfully saved as 'training_sinograms_noisefree.npy'.")

Total dataset shape: (1160, 1, 220, 220)
Data has been successfully saved as 'training_sinograms_noisefree.npy'.


In [6]:
import numpy as np

# Load the original noise-free dataset
noisefree_data = np.load('./PET_data/Train_npy/training_sinograms_noisefree.npy')

# Initialize an empty list to store noisy slices
noisy_slices = []

# Iterate over each slice, adding Poisson noise to each 2D image in the slice
for slice in noisefree_data:
    # slice has shape (1, 220, 220), so we extract the 2D image using slice[0]
    noisy_image = np.random.poisson(slice[0])  # Add Poisson noise
    noisy_slices.append(noisy_image[np.newaxis, :, :])  # Add back the channel dimension

# Convert the list of noisy slices to a NumPy array
noisy_slices_array = np.array(noisy_slices)

# Ensure the data is correct
print(f"Shape of the dataset after adding noise: {noisy_slices_array.shape}")

# Save the new dataset with added noise
np.save('./PET_data/Train_npy/training_sinograms_noisy.npy', noisy_slices_array)

print("Noise has been successfully added and saved as 'training_sinograms_noisy.npy'.")

Shape of the dataset after adding noise: (1160, 1, 220, 220)
Noise has been successfully added and saved as 'training_sinograms_noisy.npy'.


In [5]:
# create test dataset
import os
import nibabel as nib
import numpy as np

# Set the data directory
data_dir = './brainweb_petmr_v2'

# Initialize an empty list to store slice data
all_slices = []

# Iterate over each subject folder (assuming each folder starts with "subject")
for subject_folder in os.listdir(data_dir):
    subject_path = os.path.join(data_dir, subject_folder)
    if os.path.isdir(subject_path) and subject_folder.startswith('subject'):
        # For each subject, load image_0 and image_1
        for image_file in ['image_2.nii.gz']:
            image_path = os.path.join(subject_path, image_file)
            if os.path.exists(image_path):
                # Load the image file
                image = nib.load(image_path)
                image_data = image.get_fdata()  # Extract as a NumPy array
                
                # Get the center slice index
                center_slice = image_data.shape[2] // 2
                
                # Determine the range of slices (center ± 14 slices, total 35 slices)
                start_slice = max(0, center_slice - 14)
                end_slice = min(image_data.shape[2], center_slice + 15)  # Exclusive
                
                # Extract the slices and add them to all_slices
                slices = image_data[:, :, start_slice:end_slice]
                all_slices.extend([slices[:, :, i] for i in range(slices.shape[2])])

# Combine all slices into a single NumPy array
all_slices_array = np.array(all_slices)

# Add a new dimension to the data
all_slices_array = np.expand_dims(all_slices_array, axis=1)  # Shape: (N, 1, 220, 220)

# Ensure the data is correct
print(f"Total dataset shape: {all_slices_array.shape}")

# Save as a .npy file
os.makedirs('./PET_data/Test_npy', exist_ok=True)
np.save('./PET_data/Train_npy/test_sinograms_noisefree.npy', all_slices_array)

print("Data has been successfully saved as 'test_sinograms_noisefree.npy'.")

Total dataset shape: (580, 1, 220, 220)
Data has been successfully saved as 'test_sinograms_noisefree.npy'.


In [7]:
import numpy as np

# Load the original noise-free dataset
noisefree_data = np.load('./PET_data/Train_npy/test_sinograms_noisefree.npy')

# Initialize an empty list to store noisy slices
noisy_slices = []

# Iterate over each slice, adding Poisson noise to each 2D image in the slice
for slice in noisefree_data:
    # slice has shape (1, 220, 220), so we extract the 2D image using slice[0]
    noisy_image = np.random.poisson(slice[0])  # Add Poisson noise
    noisy_slices.append(noisy_image[np.newaxis, :, :])  # Add back the channel dimension

# Convert the list of noisy slices to a NumPy array
noisy_slices_array = np.array(noisy_slices)

# Ensure the data is correct
print(f"Shape of the dataset after adding noise: {noisy_slices_array.shape}")

# Save the new dataset with added noise
np.save('./PET_data/Test_npy/test_sinograms_noisy.npy', noisy_slices_array)

print("Noise has been successfully added and saved as 'test_sinograms_noisy.npy'.")

Shape of the dataset after adding noise: (580, 1, 220, 220)
Noise has been successfully added and saved as 'test_sinograms_noisy.npy'.
